In [1]:
import glob
import time 
import os 


import numpy as np
import pandas as pd


import h2o

from h2o.estimators.random_forest import H2ORandomForestEstimator
from sklearn.metrics import mean_absolute_error, mean_squared_error 

from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'hyperopt'

This is where we read the data

In [2]:
col_names= col_names = ['yyyy', 'mm', 'dd', 'HH', 'LON', 'LAT', 'Pres(hpa)', 'T(Celcius)', 'SST(Celcius)', 'RH(%)', 'u(m/s)', 'v(m/s)', 'WS(m/s)', 'WD(degrees)', 'q(g/kg)', 'dq(g/kg)', 'ASTD(Celcius)', 'RiB', 'EDH(m)']

df = pd.concat([pd.read_csv(f, header = None, names = col_names, sep = '\s+') for f in glob.glob('data/*.txt')], 
               ignore_index = True, sort = False)
              
print(df.shape)

df.head()
              

(6120516, 19)


,yyyy,mm,dd,HH,LON,LAT,Pres(hpa),T(Celcius),SST(Celcius),RH(%),u(m/s),v(m/s),WS(m/s),WD(degrees),q(g/kg),dq(g/kg),ASTD(Celcius),RiB,EDH(m)
0,2015,10,13,0,-76.7401,34.8289,1008.15,18.86,19.72,94.00,-0.06,0.81,0.82,175.97,12.63,1.26,-0.85,0.86,0.0
1,2015,10,13,1,-76.7178,34.8293,1008.20,18.78,19.72,93.98,-0.09,0.83,0.84,173.90,12.56,1.32,-0.94,1.03,0.0
2,2015,10,13,2,-76.6955,34.8297,1008.26,18.69,19.65,93.99,-0.18,0.81,0.83,167.29,12.50,1.34,-0.96,1.28,0.0
3,2015,10,13,3,-76.6732,34.8300,1008.31,18.60,19.60,94.06,-0.21,0.90,0.92,166.58,12.43,1.36,-1.00,1.58,0.0
4,2015,10,13,4,-76.6509,34.8304,1008.33,18.49,19.53,93.98,-0.25,0.91,0.95,164.80,12.34,1.39,-1.04,2.14,0.0


In [3]:
df.min()

yyyy             2.015000e+03
mm               1.000000e+01
dd               1.300000e+01
HH               0.000000e+00
LON             -7.680440e+01
LAT              3.482890e+01
Pres(hpa)        0.000000e+00
T(Celcius)      -2.731500e+02
SST(Celcius)    -2.731500e+02
RH(%)            1.786000e+01
u(m/s)          -1.304000e+01
v(m/s)          -1.438000e+01
WS(m/s)          0.000000e+00
WD(degrees)      0.000000e+00
q(g/kg)          2.160000e+00
dq(g/kg)        -4.090000e+00
ASTD(Celcius)   -1.748000e+01
RiB             -2.059190e+08
EDH(m)           0.000000e+00
dtype: float64

In [4]:
df.max()

yyyy             2.015000e+03
mm               1.000000e+01
dd               3.000000e+01
HH               2.300000e+01
LON             -7.386330e+01
LAT              3.718000e+01
Pres(hpa)        1.030210e+03
T(Celcius)       2.645000e+01
SST(Celcius)     3.334000e+01
RH(%)            1.000300e+02
u(m/s)           1.495000e+01
v(m/s)           1.473000e+01
WS(m/s)          1.750000e+01
WD(degrees)      3.600000e+02
q(g/kg)                   inf
dq(g/kg)                  inf
ASTD(Celcius)    2.971500e+02
RiB              1.510094e+06
EDH(m)           4.000000e+01
dtype: float64

In [5]:
df.isnull().any()

yyyy             False
mm               False
dd               False
HH               False
LON              False
LAT              False
Pres(hpa)        False
T(Celcius)       False
SST(Celcius)     False
RH(%)            False
u(m/s)           False
v(m/s)           False
WS(m/s)          False
WD(degrees)      False
q(g/kg)          False
dq(g/kg)          True
ASTD(Celcius)    False
RiB              False
EDH(m)           False
dtype: bool

In [7]:
df = df.replace([-np.inf, np.inf], np.nan).dropna()

In [8]:
df.max()

yyyy             2.015000e+03
mm               1.000000e+01
dd               3.000000e+01
HH               2.300000e+01
LON             -7.386330e+01
LAT              3.718000e+01
Pres(hpa)        1.030210e+03
T(Celcius)       2.645000e+01
SST(Celcius)     3.334000e+01
RH(%)            1.000300e+02
u(m/s)           1.495000e+01
v(m/s)           1.473000e+01
WS(m/s)          1.750000e+01
WD(degrees)      3.600000e+02
q(g/kg)          1.734000e+01
dq(g/kg)         2.231000e+01
ASTD(Celcius)    8.800000e+00
RiB              1.510094e+06
EDH(m)           4.000000e+01
dtype: float64

In [10]:
df.isnull().any()

yyyy             False
mm               False
dd               False
HH               False
LON              False
LAT              False
Pres(hpa)        False
T(Celcius)       False
SST(Celcius)     False
RH(%)            False
u(m/s)           False
v(m/s)           False
WS(m/s)          False
WD(degrees)      False
q(g/kg)          False
dq(g/kg)         False
ASTD(Celcius)    False
RiB              False
EDH(m)           False
dtype: bool

In [11]:
h2o.init(max_mem_size=12)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_91"; Java(TM) SE Runtime Environment (build 1.8.0_91-b14); Java HotSpot(TM) 64-Bit Server VM (build 25.91-b14, mixed mode)
  Starting server from /Users/denny/anaconda3/envs/EDH_ML/h2o_jar/h2o.jar
  Ice root: /var/folders/t_/vqhq6xfs10n48trt1p70rmf80000gr/T/tmpsv7iti6p
  JVM stdout: /var/folders/t_/vqhq6xfs10n48trt1p70rmf80000gr/T/tmpsv7iti6p/h2o_denny_started_from_python.out
  JVM stderr: /var/folders/t_/vqhq6xfs10n48trt1p70rmf80000gr/T/tmpsv7iti6p/h2o_denny_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,"1 year, 3 months and 19 days !!!"
H2O cluster name:,H2O_from_python_denny_z9tagr
H2O cluster total nodes:,1
H2O cluster free memory:,10.67 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [12]:
hf=h2o.H2OFrame(df)

/Users/denny/anaconda3/envs/EDH_ML/lib/python3.6/site-packages/h2o/utils/shared_utils.py:170: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [13]:
x = ['Pres(hpa)', 'T(Celcius)', 'SST(Celcius)', 'RH(%)', 'u(m/s)', 'v(m/s)', 'WS(m/s)', 'WD(degrees)', 'LAT', 'LON']
y = 'EDH(m)'

In [16]:
hf_train, hf_test = hf.split_frame(ratios=[0.8])

print(len(hf_train)); print(len(hf_test))

len(hf_train) + len(hf_test) == len(hf)

4738422
1184658


True

In [17]:
edh_rf = H2ORandomForestEstimator(max_depth = 50, min_rows = 1, min_split_improvement = 1e-10, ntrees = 200)

In [18]:
edh_rf.train(x=x, y=y, training_frame = hf_train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [19]:
edh_rf

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1561502481628_1


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.6995286555138215
RMSE: 0.8363782968931114
MAE: 0.1645204543366751
RMSLE: 0.12017056448805492
Mean Residual Deviance: 0.6995286555138215
Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2019-06-25 15:55:24,0.018 sec,0.0,nan,nan,nan
,2019-06-25 15:55:34,9.511 sec,1.0,1.5923461,0.2013429,2.5355661
,2019-06-25 15:55:43,18.537 sec,2.0,1.5566375,0.2018499,2.4231203
,2019-06-25 15:55:53,28.690 sec,3.0,1.4854438,0.1993319,2.2065431
,2019-06-25 15:56:02,37.288 sec,4.0,1.4146397,0.1961266,2.0012056
---,---,---,---,---,---,---
,2019-06-25 16:27:11,31 min 46.923 sec,196.0,0.8369297,0.1646271,0.7004513
,2019-06-25 16:27:21,31 min 56.941 sec,197.0,0.8368497,0.1646090,0.7003174
,2019-06-25 16:27:32,32 min 7.431 sec,198.0,0.8366873,0.1645770,0.7000456
,2019-06-25 16:27:42,32 min 17.327 sec,199.0,0.8364133,0.1645348,0.6995873



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
WS(m/s),5462645760.0000000,1.0,0.2123179
LON,4677816832.0000000,0.8563281,0.1818138
RH(%),4427348992.0000000,0.8104770,0.1720788
SST(Celcius),4125346048.0000000,0.7551919,0.1603408
T(Celcius),1753944576.0000000,0.3210797,0.0681710
Pres(hpa),1356307072.0000000,0.2482876,0.0527159
v(m/s),1279772416.0000000,0.2342770,0.0497412
u(m/s),1057642112.0000000,0.1936135,0.0411076
LAT,970531072.0000000,0.1776669,0.0377219
WD(degrees),617257280.0000000,0.1129960,0.0239911


In [20]:
edh_perf = edh_rf.model_performance(hf_test)
edh_perf


ModelMetricsRegression: drf
** Reported on test data. **

MSE: 0.6817098929027238
RMSE: 0.8256572490463121
MAE: 0.16292424563916838
RMSLE: 0.11827555691352895
Mean Residual Deviance: 0.6817098929027238
